# In Class Exercise 5: Systems of Linear Equations

1a) Write the Force equations using matrix equation of the form $ F = \hat{K}x - b$
F1 = −k1(x1 −L1) + k2(x2 −x1 −L2)
F2 = −k2(x2 −x1 −L2) + k3(x3 −x2 −L3)
F3 = −k3(x3 −x2 −L3) + k4(Lw −x3 −L4)
Solving on paper, we get the following for $\hat{K}$ and $b$:

$$
\hat{K}=
\begin{bmatrix}
-(k_1+k_2) & k_2 & 0\\
k_2 & -(k_2+k_3) & k_3\\
0 & k_3 & -(k_3 + k_4)
\end{bmatrix}
\mbox{ and }
b =
\begin{bmatrix}
k_1L_1 - k_2L_2 \\
k_2L_2 - k_3L_3 \\
k_3L_3 +k_4L_W-k_4L_4
\end{bmatrix}
$$

2a) Write a program to solve for the rest positions of the masses in equilibrium and the final lengths of the springs